# Silver Layer - Data Cleaning with MapReduce

## Lending Club Loan Data Pipeline

**Use Case:** Predict loan default risk and analyze factors affecting loan approval

This notebook implements the Silver (Cleaned) layer of the Medallion Architecture:
- Clean and transform data using **RDD MapReduce operations** (no DataFrames/SQL)
- Handle missing values, type conversions, and data standardization
- Profile and tune performance

**Important:** As per project requirements, this notebook uses basic MapReduce routines in Spark (map, filter, reduce, reduceByKey, etc.) - NOT DataFrames or SQL.

## 1. Setup and Configuration

In [1]:
import time
import json
import re
import os
from datetime import datetime
from collections import defaultdict

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *

print("Setup complete!")

# Initialize Spark Session with tuned configuration
spark = SparkSession.builder \
    .appName("LendingClub-Silver-Layer") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

print(f"Spark Version: {spark.version}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 18:52:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.5.0


In [2]:
# Define paths (matching Bronze notebook)
BRONZE_PATH = "../data/medallion/bronze/"
SILVER_PATH = "../data/medallion/silver/"

BRONZE_ACCEPTED_PATH = os.path.join(BRONZE_PATH, "accepted_loans")
BRONZE_REJECTED_PATH = os.path.join(BRONZE_PATH, "rejected_loans")

# Create silver directory
os.makedirs(SILVER_PATH, exist_ok=True)

print(f"Bronze input path: {BRONZE_PATH}")
print(f"Silver output path: {SILVER_PATH}")

Bronze input path: ../data/medallion/bronze/
Silver output path: ../data/medallion/silver/


## 2. Load Bronze Data and Convert to RDD

We load the Parquet data into a DataFrame first (for efficient reading), then immediately convert to RDD for MapReduce operations.

In [3]:
# Load Bronze data and convert to RDD
# We read parquet (efficient) then convert to RDD of Row objects

accepted_df = spark.read.parquet(BRONZE_ACCEPTED_PATH)
rejected_df = spark.read.parquet(BRONZE_REJECTED_PATH)

# Get column names for reference
accepted_columns = accepted_df.columns
rejected_columns = rejected_df.columns

print(f"Accepted loans columns: {len(accepted_columns)}")
print(f"Rejected loans columns: {len(rejected_columns)}")

# Convert to RDD - each element is a Row object
accepted_rdd = accepted_df.rdd
rejected_rdd = rejected_df.rdd

print(f"\nAccepted loans RDD partitions: {accepted_rdd.getNumPartitions()}")
print(f"Rejected loans RDD partitions: {rejected_rdd.getNumPartitions()}")

Accepted loans columns: 154
Rejected loans columns: 12

Accepted loans RDD partitions: 4
Rejected loans RDD partitions: 4


In [4]:
# Examine sample row structure
sample_row = accepted_rdd.first()
print("Sample row type:", type(sample_row))
print("\nSample row as dict (first 10 fields):")
sample_dict = sample_row.asDict()
for i, (k, v) in enumerate(sample_dict.items()):
    if i < 10:
        print(f"  {k}: {v} (type: {type(v).__name__})")

Sample row type: <class 'pyspark.sql.types.Row'>

Sample row as dict (first 10 fields):
  id: 68407277 (type: str)
  member_id: None (type: NoneType)
  loan_amnt: 3600.0 (type: str)
  funded_amnt: 3600.0 (type: str)
  funded_amnt_inv: 3600.0 (type: str)
  term:  36 months (type: str)
  int_rate: 13.99 (type: str)
  installment: 123.03 (type: str)
  grade: C (type: str)
  sub_grade: C4 (type: str)


## 3. Identify Data Quality Issues

Before cleaning, let's identify the specific issues in the data using MapReduce operations.

In [5]:
# Select key columns for our use case (loan default prediction)
# These are the columns we'll clean and use in the Gold layer

KEY_COLUMNS = [
    # Loan characteristics
    'loan_amnt',        # Loan amount
    'term',             # Loan term (36 or 60 months)
    'int_rate',         # Interest rate
    'installment',      # Monthly installment
    'grade',            # Loan grade (A-G)
    'sub_grade',        # Loan sub-grade (A1-G5)
    
    # Borrower information
    'emp_length',       # Employment length
    'home_ownership',   # Home ownership status
    'annual_inc',       # Annual income
    'verification_status',  # Income verification
    
    # Loan purpose and status
    'purpose',          # Loan purpose
    'loan_status',      # Current loan status (TARGET)
    'issue_d',          # Issue date
    
    # Credit history
    'dti',              # Debt-to-income ratio
    'earliest_cr_line', # Earliest credit line
    'open_acc',         # Open credit accounts
    'pub_rec',          # Public records
    'revol_bal',        # Revolving balance
    'revol_util',       # Revolving utilization
    'total_acc',        # Total accounts
    'fico_range_low',   # FICO score low
    'fico_range_high',  # FICO score high
    
    # Additional useful features
    'addr_state',       # State
    'delinq_2yrs',      # Delinquencies in 2 years
    'inq_last_6mths',   # Inquiries in last 6 months
    'mort_acc',         # Mortgage accounts
    'pub_rec_bankruptcies'  # Bankruptcies
]

print(f"Selected {len(KEY_COLUMNS)} key columns for cleaning")

Selected 27 key columns for cleaning


In [6]:
%%time
# Profile null values using MapReduce
# Map: emit (column_name, 1) if value is null/empty, else (column_name, 0)
# Reduce: sum to get total nulls per column

def count_nulls_mapper(row):
    """Map function to count nulls for each column"""
    row_dict = row.asDict()
    results = []
    for col in KEY_COLUMNS:
        if col in row_dict:
            value = row_dict[col]
            # Check for null, None, empty string, or 'null' string
            is_null = (value is None or 
                      value == '' or 
                      str(value).lower() == 'null' or
                      str(value).lower() == 'nan')
            results.append((col, 1 if is_null else 0))
    return results

# Use flatMap since mapper returns multiple pairs
null_counts = accepted_rdd \
    .flatMap(count_nulls_mapper) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

total_rows = accepted_rdd.count()

print(f"Total rows: {total_rows:,}")
print("\nNull/Missing values per column:")
print("-" * 50)
for col, count in sorted(null_counts, key=lambda x: x[1], reverse=True):
    pct = (count / total_rows) * 100
    print(f"{col:25s}: {count:>10,} ({pct:5.2f}%)")

Total rows: 2,260,701

Null/Missing values per column:
--------------------------------------------------
emp_length               :    146,940 ( 6.50%)
mort_acc                 :     50,063 ( 2.21%)
revol_util               :      1,835 ( 0.08%)
dti                      :      1,744 ( 0.08%)
pub_rec_bankruptcies     :      1,398 ( 0.06%)
inq_last_6mths           :         63 ( 0.00%)
open_acc                 :         62 ( 0.00%)
earliest_cr_line         :         62 ( 0.00%)
total_acc                :         62 ( 0.00%)
pub_rec                  :         62 ( 0.00%)
delinq_2yrs              :         62 ( 0.00%)
annual_inc               :         37 ( 0.00%)
grade                    :         33 ( 0.00%)
loan_status              :         33 ( 0.00%)
issue_d                  :         33 ( 0.00%)
fico_range_high          :         33 ( 0.00%)
addr_state               :         33 ( 0.00%)
installment              :         33 ( 0.00%)
purpose                  :         33 ( 0.00%)
t

In [7]:
%%time
# Profile unique values for categorical columns using MapReduce

categorical_cols = ['term', 'grade', 'sub_grade', 'emp_length', 
                    'home_ownership', 'verification_status', 'purpose', 'loan_status']

def extract_categorical_mapper(row):
    """Extract categorical column values"""
    row_dict = row.asDict()
    results = []
    for col in categorical_cols:
        if col in row_dict:
            value = str(row_dict[col]) if row_dict[col] is not None else 'NULL'
            results.append(((col, value), 1))
    return results

# Count occurrences of each value per column
categorical_counts = accepted_rdd \
    .flatMap(extract_categorical_mapper) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

# Organize by column
cat_summary = defaultdict(dict)
for (col, value), count in categorical_counts:
    cat_summary[col][value] = count

# Display
for col in categorical_cols:
    print(f"\n{col}:")
    values = cat_summary[col]
    for value, count in sorted(values.items(), key=lambda x: x[1], reverse=True)[:10]:
        pct = (count / total_rows) * 100
        print(f"  {value:30s}: {count:>10,} ({pct:5.2f}%)")


term:
   36 months                    :  1,609,754 (71.21%)
   60 months                    :    650,914 (28.79%)
  NULL                          :         33 ( 0.00%)

grade:
  B                             :    663,557 (29.35%)
  C                             :    650,053 (28.75%)
  A                             :    433,027 (19.15%)
  D                             :    324,424 (14.35%)
  E                             :    135,639 ( 6.00%)
  F                             :     41,800 ( 1.85%)
  G                             :     12,168 ( 0.54%)
  NULL                          :         33 ( 0.00%)

sub_grade:
  C1                            :    145,903 ( 6.45%)
  B5                            :    140,288 ( 6.21%)
  B4                            :    139,793 ( 6.18%)
  B3                            :    131,514 ( 5.82%)
  C2                            :    131,116 ( 5.80%)
  C3                            :    129,193 ( 5.71%)
  C4                            :    127,115 ( 5.62%)
 

## 4. Define Cleaning Functions

Now we define the cleaning functions that will be applied via MapReduce operations.

In [8]:
# Cleaning utility functions

def clean_percentage(value):
    """Clean percentage values like '13.99%' -> 13.99"""
    if value is None or value == '':
        return None
    try:
        cleaned = str(value).replace('%', '').strip()
        return float(cleaned)
    except (ValueError, TypeError):
        return None

def clean_currency(value):
    """Clean currency values like '$1,234.56' -> 1234.56"""
    if value is None or value == '':
        return None
    try:
        cleaned = str(value).replace('$', '').replace(',', '').strip()
        return float(cleaned)
    except (ValueError, TypeError):
        return None

def clean_term(value):
    """Clean term values like ' 36 months' -> 36"""
    if value is None or value == '':
        return None
    try:
        # Extract numeric part
        match = re.search(r'(\d+)', str(value))
        if match:
            return int(match.group(1))
        return None
    except (ValueError, TypeError):
        return None

def clean_emp_length(value):
    """Clean employment length:
    '10+ years' -> 10
    '< 1 year' -> 0
    '5 years' -> 5
    'n/a' -> None
    """
    if value is None or value == '' or str(value).lower() == 'n/a':
        return None
    value_str = str(value).lower()
    if '10+' in value_str:
        return 10
    if '< 1' in value_str:
        return 0
    try:
        match = re.search(r'(\d+)', value_str)
        if match:
            return int(match.group(1))
        return None
    except (ValueError, TypeError):
        return None

def clean_numeric(value):
    """Clean generic numeric values"""
    if value is None or value == '' or str(value).lower() in ['null', 'nan', 'none']:
        return None
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def clean_date(value):
    """Clean date values like 'Dec-2015' -> '2015-12-01'"""
    if value is None or value == '':
        return None
    try:
        # Parse format like 'Dec-2015'
        dt = datetime.strptime(str(value), '%b-%Y')
        return dt.strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        try:
            # Try alternate format 'Dec-15'
            dt = datetime.strptime(str(value), '%b-%y')
            return dt.strftime('%Y-%m-%d')
        except:
            return None

def clean_string(value):
    """Clean string values - strip whitespace, standardize nulls"""
    if value is None or str(value).lower() in ['null', 'nan', 'none', '']:
        return None
    return str(value).strip()

# Test cleaning functions
print("Testing cleaning functions:")
print(f"  clean_percentage('13.99%') = {clean_percentage('13.99%')}")
print(f"  clean_term(' 36 months') = {clean_term(' 36 months')}")
print(f"  clean_emp_length('10+ years') = {clean_emp_length('10+ years')}")
print(f"  clean_emp_length('< 1 year') = {clean_emp_length('< 1 year')}")
print(f"  clean_emp_length('5 years') = {clean_emp_length('5 years')}")
print(f"  clean_date('Dec-2015') = {clean_date('Dec-2015')}")

Testing cleaning functions:
  clean_percentage('13.99%') = 13.99
  clean_term(' 36 months') = 36
  clean_emp_length('10+ years') = 10
  clean_emp_length('< 1 year') = 0
  clean_emp_length('5 years') = 5
  clean_date('Dec-2015') = 2015-12-01


## 5. Apply Cleaning via MapReduce

This is the core cleaning step using **map** transformation.

In [9]:
def create_loan_status_binary(loan_status):
    """
    Create binary target variable for loan default prediction.
    1 = Default (bad loan)
    0 = Paid/Current (good loan)
    None = Unknown/Exclude
    """
    if loan_status is None:
        return None
    
    status = str(loan_status).lower().strip()
    
    # Bad loans (default = 1)
    bad_statuses = ['charged off', 'default', 'late (31-120 days)', 
                   'late (16-30 days)', 'does not meet the credit policy. status:charged off']
    
    # Good loans (default = 0)
    good_statuses = ['fully paid', 'current', 
                    'does not meet the credit policy. status:fully paid',
                    'in grace period']
    
    if status in bad_statuses:
        return 1
    elif status in good_statuses:
        return 0
    else:
        return None  # Exclude unclear statuses

In [10]:
def clean_accepted_loan_row(row):
    """
    Main cleaning function applied to each row via map().
    Returns a dictionary with cleaned values.
    """
    row_dict = row.asDict()
    
    cleaned = {
        # Loan characteristics
        'loan_amnt': clean_numeric(row_dict.get('loan_amnt')),
        'term': clean_term(row_dict.get('term')),
        'int_rate': clean_percentage(row_dict.get('int_rate')) if '%' in str(row_dict.get('int_rate', '')) else clean_numeric(row_dict.get('int_rate')),
        'installment': clean_numeric(row_dict.get('installment')),
        'grade': clean_string(row_dict.get('grade')),
        'sub_grade': clean_string(row_dict.get('sub_grade')),
        
        # Borrower information
        'emp_length': clean_emp_length(row_dict.get('emp_length')),
        'home_ownership': clean_string(row_dict.get('home_ownership')),
        'annual_inc': clean_numeric(row_dict.get('annual_inc')),
        'verification_status': clean_string(row_dict.get('verification_status')),
        
        # Loan purpose and status
        'purpose': clean_string(row_dict.get('purpose')),
        'loan_status': clean_string(row_dict.get('loan_status')),
        'loan_status_binary': create_loan_status_binary(row_dict.get('loan_status')),
        'issue_d': clean_date(row_dict.get('issue_d')),
        
        # Credit history
        'dti': clean_numeric(row_dict.get('dti')),
        'earliest_cr_line': clean_date(row_dict.get('earliest_cr_line')),
        'open_acc': clean_numeric(row_dict.get('open_acc')),
        'pub_rec': clean_numeric(row_dict.get('pub_rec')),
        'revol_bal': clean_numeric(row_dict.get('revol_bal')),
        'revol_util': clean_percentage(row_dict.get('revol_util')) if '%' in str(row_dict.get('revol_util', '')) else clean_numeric(row_dict.get('revol_util')),
        'total_acc': clean_numeric(row_dict.get('total_acc')),
        'fico_range_low': clean_numeric(row_dict.get('fico_range_low')),
        'fico_range_high': clean_numeric(row_dict.get('fico_range_high')),
        
        # Additional features
        'addr_state': clean_string(row_dict.get('addr_state')),
        'delinq_2yrs': clean_numeric(row_dict.get('delinq_2yrs')),
        'inq_last_6mths': clean_numeric(row_dict.get('inq_last_6mths')),
        'mort_acc': clean_numeric(row_dict.get('mort_acc')),
        'pub_rec_bankruptcies': clean_numeric(row_dict.get('pub_rec_bankruptcies')),
    }
    
    # Calculate derived features
    # FICO average
    if cleaned['fico_range_low'] and cleaned['fico_range_high']:
        cleaned['fico_avg'] = (cleaned['fico_range_low'] + cleaned['fico_range_high']) / 2
    else:
        cleaned['fico_avg'] = None
    
    # Loan to income ratio
    if cleaned['loan_amnt'] and cleaned['annual_inc'] and cleaned['annual_inc'] > 0:
        cleaned['loan_to_income'] = cleaned['loan_amnt'] / cleaned['annual_inc']
    else:
        cleaned['loan_to_income'] = None
    
    return cleaned

# Test on sample row
test_cleaned = clean_accepted_loan_row(sample_row)
print("Cleaned sample row:")
for k, v in test_cleaned.items():
    print(f"  {k}: {v}")

Cleaned sample row:
  loan_amnt: 3600.0
  term: 36
  int_rate: 13.99
  installment: 123.03
  grade: C
  sub_grade: C4
  emp_length: 10
  home_ownership: MORTGAGE
  annual_inc: 55000.0
  verification_status: Not Verified
  purpose: debt_consolidation
  loan_status: Fully Paid
  loan_status_binary: 0
  issue_d: 2015-12-01
  dti: 5.91
  earliest_cr_line: 2003-08-01
  open_acc: 7.0
  pub_rec: 0.0
  revol_bal: 2765.0
  revol_util: 29.7
  total_acc: 13.0
  fico_range_low: 675.0
  fico_range_high: 679.0
  addr_state: PA
  delinq_2yrs: 0.0
  inq_last_6mths: 1.0
  mort_acc: 1.0
  pub_rec_bankruptcies: 0.0
  fico_avg: 677.0
  loan_to_income: 0.06545454545454546


In [11]:
%%time
# Apply cleaning transformation using map()
# This is the core MapReduce cleaning operation

print("Applying cleaning transformation via map()...")

cleaned_rdd = accepted_rdd.map(clean_accepted_loan_row)

# Cache for reuse (important for performance)
cleaned_rdd.cache()

# Force evaluation and count
cleaned_count = cleaned_rdd.count()
print(f"Cleaned RDD count: {cleaned_count:,}")

Applying cleaning transformation via map()...


Cleaned RDD count: 2,260,701
CPU times: user 21.7 ms, sys: 9.74 ms, total: 31.4 ms
Wall time: 1min 3s


## 6. Filter Invalid Records

Using **filter** transformation to remove invalid records.

In [12]:
def is_valid_loan_record(row_dict):
    """
    Filter function to identify valid records.
    Returns True if record should be kept.
    """
    # Must have loan amount
    if row_dict.get('loan_amnt') is None or row_dict['loan_amnt'] <= 0:
        return False
    
    # Must have a determinable loan status for ML
    if row_dict.get('loan_status_binary') is None:
        return False
    
    # Must have interest rate
    if row_dict.get('int_rate') is None or row_dict['int_rate'] <= 0:
        return False
    
    # Must have grade
    if row_dict.get('grade') is None:
        return False
    
    # Must have annual income (and it should be positive)
    if row_dict.get('annual_inc') is None or row_dict['annual_inc'] <= 0:
        return False
    
    # Reasonable bounds check
    # Interest rate should be between 0 and 50%
    if row_dict['int_rate'] > 50:
        return False
    
    # Annual income should be reasonable (< $10M)
    if row_dict['annual_inc'] > 10000000:
        return False
    
    return True

In [13]:
%%time
# Apply filter transformation
print("Applying filter transformation...")

filtered_rdd = cleaned_rdd.filter(is_valid_loan_record)

# Cache filtered RDD
filtered_rdd.cache()

filtered_count = filtered_rdd.count()
removed_count = cleaned_count - filtered_count

print(f"Records after filtering: {filtered_count:,}")
print(f"Records removed: {removed_count:,} ({removed_count/cleaned_count*100:.2f}%)")

Applying filter transformation...


Records after filtering: 2,258,994
Records removed: 1,707 (0.08%)
CPU times: user 11.9 ms, sys: 6.01 ms, total: 17.9 ms
Wall time: 5.26 s


## 7. Profile Cleaned Data with MapReduce Aggregations

Using **reduceByKey**, **aggregateByKey**, and other MapReduce operations to profile the cleaned data.

In [14]:
%%time
# Compute statistics using MapReduce
# Count by loan_status_binary using map + reduceByKey

status_counts = filtered_rdd \
    .map(lambda x: (x['loan_status_binary'], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

print("Loan Status Distribution:")
for status, count in sorted(status_counts):
    label = "Default" if status == 1 else "Paid/Current"
    pct = count / filtered_count * 100
    print(f"  {label} ({status}): {count:>10,} ({pct:5.2f}%)")

Loan Status Distribution:
  Paid/Current (0):  1,963,912 (86.94%)
  Default (1):    295,082 (13.06%)
CPU times: user 10.6 ms, sys: 3.02 ms, total: 13.7 ms
Wall time: 2.03 s


In [15]:
%%time
# Grade distribution using MapReduce

grade_counts = filtered_rdd \
    .map(lambda x: (x['grade'], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

print("Grade Distribution:")
for grade, count in sorted(grade_counts):
    pct = count / filtered_count * 100
    print(f"  Grade {grade}: {count:>10,} ({pct:5.2f}%)")

Grade Distribution:
  Grade A:    432,747 (19.16%)
  Grade B:    663,130 (29.36%)
  Grade C:    649,552 (28.75%)
  Grade D:    324,099 (14.35%)
  Grade E:    135,542 ( 6.00%)
  Grade F:     41,773 ( 1.85%)
  Grade G:     12,151 ( 0.54%)
CPU times: user 11.6 ms, sys: 4.9 ms, total: 16.5 ms
Wall time: 2 s


In [16]:
%%time
# Default rate by grade using MapReduce
# Map: (grade, (default_status, 1))
# Reduce: (grade, (total_defaults, total_count))

def grade_default_mapper(row):
    grade = row['grade']
    is_default = row['loan_status_binary']
    return (grade, (is_default, 1))

def grade_default_reducer(a, b):
    return (a[0] + b[0], a[1] + b[1])

default_by_grade = filtered_rdd \
    .map(grade_default_mapper) \
    .reduceByKey(grade_default_reducer) \
    .collect()

print("Default Rate by Grade:")
print("-" * 50)
for grade, (defaults, total) in sorted(default_by_grade):
    rate = defaults / total * 100
    print(f"  Grade {grade}: {rate:5.2f}% default rate ({defaults:,} / {total:,})")

Default Rate by Grade:
--------------------------------------------------
  Grade A:  3.67% default rate (15,876 / 432,747)
  Grade B:  8.81% default rate (58,429 / 663,130)
  Grade C: 14.59% default rate (94,795 / 649,552)
  Grade D: 20.66% default rate (66,970 / 324,099)
  Grade E: 28.60% default rate (38,760 / 135,542)
  Grade F: 36.74% default rate (15,349 / 41,773)
  Grade G: 40.35% default rate (4,903 / 12,151)
CPU times: user 13.3 ms, sys: 3.17 ms, total: 16.5 ms
Wall time: 2.03 s


In [17]:
%%time
# Compute basic statistics for numeric columns using aggregate
# Using aggregate to compute min, max, sum, count in one pass

def stats_seq_op(acc, row):
    """Sequential operation: update accumulator with new row"""
    value = row.get('loan_amnt')
    if value is not None:
        min_val = min(acc[0], value) if acc[0] is not None else value
        max_val = max(acc[1], value) if acc[1] is not None else value
        sum_val = acc[2] + value
        count_val = acc[3] + 1
        return (min_val, max_val, sum_val, count_val)
    return acc

def stats_comb_op(acc1, acc2):
    """Combine operation: merge two accumulators"""
    min_val = min(acc1[0], acc2[0]) if acc1[0] is not None and acc2[0] is not None else acc1[0] or acc2[0]
    max_val = max(acc1[1], acc2[1]) if acc1[1] is not None and acc2[1] is not None else acc1[1] or acc2[1]
    sum_val = acc1[2] + acc2[2]
    count_val = acc1[3] + acc2[3]
    return (min_val, max_val, sum_val, count_val)

# Initial accumulator: (min, max, sum, count)
zero_value = (None, None, 0.0, 0)

loan_amnt_stats = filtered_rdd.aggregate(zero_value, stats_seq_op, stats_comb_op)

min_val, max_val, sum_val, count_val = loan_amnt_stats
avg_val = sum_val / count_val if count_val > 0 else 0

print("Loan Amount Statistics:")
print(f"  Min: ${min_val:,.2f}")
print(f"  Max: ${max_val:,.2f}")
print(f"  Avg: ${avg_val:,.2f}")
print(f"  Count: {count_val:,}")

Loan Amount Statistics:
  Min: $500.00
  Max: $40,000.00
  Avg: $15,043.90
  Count: 2,258,994
CPU times: user 11.9 ms, sys: 3.27 ms, total: 15.2 ms
Wall time: 1.87 s


In [18]:
%%time
# Compute stats for multiple columns using a single pass with aggregate

numeric_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_avg']

def multi_stats_seq_op(acc, row):
    """Update stats for all numeric columns"""
    result = dict(acc)
    for col in numeric_cols:
        value = row.get(col)
        if value is not None:
            stats = result[col]
            min_val = min(stats[0], value) if stats[0] is not None else value
            max_val = max(stats[1], value) if stats[1] is not None else value
            sum_val = stats[2] + value
            count_val = stats[3] + 1
            result[col] = (min_val, max_val, sum_val, count_val)
    return result

def multi_stats_comb_op(acc1, acc2):
    """Combine stats from two accumulators"""
    result = {}
    for col in numeric_cols:
        s1, s2 = acc1[col], acc2[col]
        min_val = min(s1[0], s2[0]) if s1[0] is not None and s2[0] is not None else s1[0] or s2[0]
        max_val = max(s1[1], s2[1]) if s1[1] is not None and s2[1] is not None else s1[1] or s2[1]
        sum_val = s1[2] + s2[2]
        count_val = s1[3] + s2[3]
        result[col] = (min_val, max_val, sum_val, count_val)
    return result

zero_multi = {col: (None, None, 0.0, 0) for col in numeric_cols}

multi_stats = filtered_rdd.aggregate(zero_multi, multi_stats_seq_op, multi_stats_comb_op)

print("Numeric Column Statistics:")
print("=" * 70)
print(f"{'Column':<15} {'Min':>12} {'Max':>12} {'Avg':>12} {'Count':>12}")
print("-" * 70)
for col in numeric_cols:
    min_v, max_v, sum_v, count_v = multi_stats[col]
    avg_v = sum_v / count_v if count_v > 0 else 0
    print(f"{col:<15} {min_v:>12,.2f} {max_v:>12,.2f} {avg_v:>12,.2f} {count_v:>12,}")

Numeric Column Statistics:
Column                   Min          Max          Avg        Count
----------------------------------------------------------------------
loan_amnt             500.00    40,000.00    15,043.90    2,258,994
int_rate                5.31        30.99        13.09    2,258,994
annual_inc              0.36 9,930,475.00    77,969.52    2,258,994
dti                    -1.00       999.00        18.82    2,258,942
fico_avg              612.00       847.50       700.58    2,258,994
CPU times: user 8.83 ms, sys: 5.15 ms, total: 14 ms
Wall time: 2.96 s


## 8. Performance Profiling and Tuning

Let's profile and tune our MapReduce operations.

In [19]:
# Option A: Skip partition size analysis entirely
print("=== Partition Analysis ===")
print(f"Original RDD partitions: {accepted_rdd.getNumPartitions()}")
print(f"Cleaned RDD partitions: {cleaned_rdd.getNumPartitions()}")
print(f"Filtered RDD partitions: {filtered_rdd.getNumPartitions()}")

# Skip glom() - it's too memory intensive
print(f"\nTotal records: {filtered_rdd.count():,}")
print(f"Approx records per partition: {filtered_rdd.count() // filtered_rdd.getNumPartitions():,}")

=== Partition Analysis ===
Original RDD partitions: 4
Cleaned RDD partitions: 4
Filtered RDD partitions: 4



Total records: 2,258,994


Approx records per partition: 564,748


In [20]:
%%time
# Experiment with different partition counts
# Find optimal partition count based on data size

# Rule of thumb: 2-4 partitions per CPU core, or ~128MB per partition
# For ~2M records, let's try different values

import time

partition_tests = [4, 8, 16, 32]
results = []

for num_partitions in partition_tests:
    # Repartition
    test_rdd = filtered_rdd.repartition(num_partitions)
    
    # Time a simple operation
    start = time.time()
    
    # Perform a MapReduce operation
    _ = test_rdd \
        .map(lambda x: (x['grade'], x['loan_amnt'])) \
        .reduceByKey(lambda a, b: a + b) \
        .collect()
    
    elapsed = time.time() - start
    results.append((num_partitions, elapsed))
    print(f"Partitions: {num_partitions:2d} | Time: {elapsed:.3f}s")

# Find best
best = min(results, key=lambda x: x[1])
print(f"\nOptimal partition count: {best[0]} (time: {best[1]:.3f}s)")

Partitions:  4 | Time: 5.887s


Partitions:  8 | Time: 5.728s


Partitions: 16 | Time: 6.398s


Partitions: 32 | Time: 6.847s

Optimal partition count: 8 (time: 5.728s)
CPU times: user 58.2 ms, sys: 22.1 ms, total: 80.3 ms
Wall time: 24.9 s


In [21]:
# Apply optimal partitioning
OPTIMAL_PARTITIONS = 8  # Adjust based on results above

optimized_rdd = filtered_rdd.repartition(OPTIMAL_PARTITIONS)
optimized_rdd.cache()

# Force evaluation
_ = optimized_rdd.count()

print(f"Optimized RDD partitions: {optimized_rdd.getNumPartitions()}")

Optimized RDD partitions: 8


## 9. Process Rejected Loans (Simplified)

Apply similar cleaning to rejected loans dataset.

In [22]:
# Check rejected loans structure
rejected_sample = rejected_rdd.first()
print("Rejected loans columns:")
for k, v in rejected_sample.asDict().items():
    print(f"  {k}: {v}")

Rejected loans columns:
  Amount Requested: 1000.0
  Application Date: 2007-05-26
  Loan Title: Wedding Covered but No Honeymoon
  Risk_Score: 693.0
  Debt-To-Income Ratio: 10%
  Zip Code: 481xx
  State: NM
  Employment Length: 4 years
  Policy Code: 0.0
  _ingestion_timestamp: 2025-11-25 15:47:47.261000
  _source_file: rejected_2007_to_2018Q4.csv
  _data_source: lending_club


In [23]:
def clean_rejected_loan_row(row):
    """
    Clean rejected loan records using map().
    """
    row_dict = row.asDict()
    
    cleaned = {
        'Amount Requested': clean_numeric(row_dict.get('Amount Requested')),
        'Application Date': clean_string(row_dict.get('Application Date')),
        'Loan Title': clean_string(row_dict.get('Loan Title')),
        'Risk_Score': clean_numeric(row_dict.get('Risk_Score')),
        'Debt-To-Income Ratio': clean_percentage(row_dict.get('Debt-To-Income Ratio')),
        'Zip Code': clean_string(row_dict.get('Zip Code')),
        'State': clean_string(row_dict.get('State')),
        'Employment Length': clean_emp_length(row_dict.get('Employment Length')),
        'Policy Code': clean_string(row_dict.get('Policy Code')),
    }
    
    return cleaned

def is_valid_rejected_record(row_dict):
    """Filter valid rejected records"""
    if row_dict.get('Amount Requested') is None or row_dict['Amount Requested'] <= 0:
        return False
    return True

In [24]:
%%time
# Clean rejected loans using MapReduce

print("Cleaning rejected loans...")

cleaned_rejected_rdd = rejected_rdd \
    .map(clean_rejected_loan_row) \
    .filter(is_valid_rejected_record)

cleaned_rejected_rdd.cache()

rejected_count = cleaned_rejected_rdd.count()
print(f"Cleaned rejected loans: {rejected_count:,}")

Cleaning rejected loans...


Cleaned rejected loans: 27,647,453
CPU times: user 27.7 ms, sys: 10.3 ms, total: 37.9 ms
Wall time: 1min 52s


## 10. Save to Silver Layer

Convert cleaned RDDs to DataFrames and save as Parquet.

In [25]:
# Define schema for accepted loans Silver layer
accepted_silver_schema = StructType([
    StructField("loan_amnt", FloatType(), True),
    StructField("term", IntegerType(), True),
    StructField("int_rate", FloatType(), True),
    StructField("installment", FloatType(), True),
    StructField("grade", StringType(), True),
    StructField("sub_grade", StringType(), True),
    StructField("emp_length", IntegerType(), True),
    StructField("home_ownership", StringType(), True),
    StructField("annual_inc", FloatType(), True),
    StructField("verification_status", StringType(), True),
    StructField("purpose", StringType(), True),
    StructField("loan_status", StringType(), True),
    StructField("loan_status_binary", IntegerType(), True),
    StructField("issue_d", StringType(), True),
    StructField("dti", FloatType(), True),
    StructField("earliest_cr_line", StringType(), True),
    StructField("open_acc", FloatType(), True),
    StructField("pub_rec", FloatType(), True),
    StructField("revol_bal", FloatType(), True),
    StructField("revol_util", FloatType(), True),
    StructField("total_acc", FloatType(), True),
    StructField("fico_range_low", FloatType(), True),
    StructField("fico_range_high", FloatType(), True),
    StructField("addr_state", StringType(), True),
    StructField("delinq_2yrs", FloatType(), True),
    StructField("inq_last_6mths", FloatType(), True),
    StructField("mort_acc", FloatType(), True),
    StructField("pub_rec_bankruptcies", FloatType(), True),
    StructField("fico_avg", FloatType(), True),
    StructField("loan_to_income", FloatType(), True),
])

In [26]:
# Convert RDD of dicts to RDD of Rows
from pyspark.sql import Row

def dict_to_row(d):
    """Convert dictionary to Row with proper type handling"""
    return Row(
        loan_amnt=float(d['loan_amnt']) if d['loan_amnt'] is not None else None,
        term=int(d['term']) if d['term'] is not None else None,
        int_rate=float(d['int_rate']) if d['int_rate'] is not None else None,
        installment=float(d['installment']) if d['installment'] is not None else None,
        grade=d['grade'],
        sub_grade=d['sub_grade'],
        emp_length=int(d['emp_length']) if d['emp_length'] is not None else None,
        home_ownership=d['home_ownership'],
        annual_inc=float(d['annual_inc']) if d['annual_inc'] is not None else None,
        verification_status=d['verification_status'],
        purpose=d['purpose'],
        loan_status=d['loan_status'],
        loan_status_binary=int(d['loan_status_binary']) if d['loan_status_binary'] is not None else None,
        issue_d=d['issue_d'],
        dti=float(d['dti']) if d['dti'] is not None else None,
        earliest_cr_line=d['earliest_cr_line'],
        open_acc=float(d['open_acc']) if d['open_acc'] is not None else None,
        pub_rec=float(d['pub_rec']) if d['pub_rec'] is not None else None,
        revol_bal=float(d['revol_bal']) if d['revol_bal'] is not None else None,
        revol_util=float(d['revol_util']) if d['revol_util'] is not None else None,
        total_acc=float(d['total_acc']) if d['total_acc'] is not None else None,
        fico_range_low=float(d['fico_range_low']) if d['fico_range_low'] is not None else None,
        fico_range_high=float(d['fico_range_high']) if d['fico_range_high'] is not None else None,
        addr_state=d['addr_state'],
        delinq_2yrs=float(d['delinq_2yrs']) if d['delinq_2yrs'] is not None else None,
        inq_last_6mths=float(d['inq_last_6mths']) if d['inq_last_6mths'] is not None else None,
        mort_acc=float(d['mort_acc']) if d['mort_acc'] is not None else None,
        pub_rec_bankruptcies=float(d['pub_rec_bankruptcies']) if d['pub_rec_bankruptcies'] is not None else None,
        fico_avg=float(d['fico_avg']) if d['fico_avg'] is not None else None,
        loan_to_income=float(d['loan_to_income']) if d['loan_to_income'] is not None else None,
    )

# Transform using map
row_rdd = optimized_rdd.map(dict_to_row)

# Create DataFrame
accepted_silver_df = spark.createDataFrame(row_rdd, schema=accepted_silver_schema)

print(f"Silver DataFrame created with {len(accepted_silver_df.columns)} columns")
accepted_silver_df.printSchema()

Silver DataFrame created with 30 columns
root
 |-- loan_amnt: float (nullable = true)
 |-- term: integer (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_status_binary: integer (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- fico_range_low: float (nullable = true)
 |-- fico_range_high: float (nullable = 

In [27]:
# Preview silver data
accepted_silver_df.select(
    'loan_amnt', 'int_rate', 'grade', 'annual_inc', 
    'loan_status_binary', 'fico_avg', 'loan_to_income'
).show(10, truncate=False)

+---------+--------+-----+----------+------------------+--------+--------------+
|loan_amnt|int_rate|grade|annual_inc|loan_status_binary|fico_avg|loan_to_income|
+---------+--------+-----+----------+------------------+--------+--------------+
|16000.0  |13.44   |C    |42000.0   |1                 |672.0   |0.3809524     |
|21875.0  |11.99   |C    |47590.0   |0                 |702.0   |0.4596554     |
|14025.0  |18.49   |D    |39000.0   |0                 |677.0   |0.3596154     |
|35000.0  |12.88   |C    |106000.0  |1                 |732.0   |0.3301887     |
|20000.0  |12.88   |C    |145000.0  |0                 |697.0   |0.13793103    |
|14000.0  |14.85   |C    |60000.0   |1                 |692.0   |0.23333333    |
|16000.0  |8.49    |B    |62000.0   |0                 |722.0   |0.2580645     |
|28000.0  |14.85   |C    |85000.0   |0                 |672.0   |0.32941177    |
|20000.0  |13.99   |C    |70000.0   |1                 |682.0   |0.2857143     |
|17600.0  |19.89   |E    |44

In [28]:
%%time
# Save to Silver layer
SILVER_ACCEPTED_PATH = os.path.join(SILVER_PATH, "accepted_loans")

accepted_silver_df.write \
    .mode("overwrite") \
    .parquet(SILVER_ACCEPTED_PATH)

print(f"Silver data saved to: {SILVER_ACCEPTED_PATH}")

Silver data saved to: ../data/medallion/silver/accepted_loans
CPU times: user 10.2 ms, sys: 2.71 ms, total: 12.9 ms
Wall time: 14.6 s


In [29]:
%%time
# Save rejected loans Silver layer
rejected_silver_schema = StructType([
    StructField("amount_requested", FloatType(), True),
    StructField("application_date", StringType(), True),
    StructField("loan_title", StringType(), True),
    StructField("risk_score", FloatType(), True),
    StructField("dti", FloatType(), True),
    StructField("zip_code", StringType(), True),
    StructField("state", StringType(), True),
    StructField("emp_length", IntegerType(), True),
    StructField("policy_code", StringType(), True),
])

def rejected_dict_to_row(d):
    return Row(
        amount_requested=float(d['Amount Requested']) if d['Amount Requested'] is not None else None,
        application_date=d['Application Date'],
        loan_title=d['Loan Title'],
        risk_score=float(d['Risk_Score']) if d['Risk_Score'] is not None else None,
        dti=float(d['Debt-To-Income Ratio']) if d['Debt-To-Income Ratio'] is not None else None,
        zip_code=d['Zip Code'],
        state=d['State'],
        emp_length=int(d['Employment Length']) if d['Employment Length'] is not None else None,
        policy_code=d['Policy Code'],
    )

rejected_row_rdd = cleaned_rejected_rdd.map(rejected_dict_to_row)
rejected_silver_df = spark.createDataFrame(rejected_row_rdd, schema=rejected_silver_schema)

SILVER_REJECTED_PATH = os.path.join(SILVER_PATH, "rejected_loans")
rejected_silver_df.write \
    .mode("overwrite") \
    .parquet(SILVER_REJECTED_PATH)

print(f"Rejected Silver data saved to: {SILVER_REJECTED_PATH}")

Rejected Silver data saved to: ../data/medallion/silver/rejected_loans
CPU times: user 18.4 ms, sys: 12 ms, total: 30.4 ms
Wall time: 1min 22s


## 11. Verification and Summary

In [30]:
# Verify saved data
verify_accepted = spark.read.parquet(SILVER_ACCEPTED_PATH)
verify_rejected = spark.read.parquet(SILVER_REJECTED_PATH)

print("=== Silver Layer Verification ===")
print(f"Accepted loans: {verify_accepted.count():,} rows, {len(verify_accepted.columns)} columns")
print(f"Rejected loans: {verify_rejected.count():,} rows, {len(verify_rejected.columns)} columns")

=== Silver Layer Verification ===
Accepted loans: 2,258,994 rows, 30 columns
Rejected loans: 27,647,453 rows, 9 columns


In [31]:
# Check storage sizes
!du -sh {SILVER_ACCEPTED_PATH}
!du -sh {SILVER_REJECTED_PATH}

61M	../data/medallion/silver/accepted_loans
211M	../data/medallion/silver/rejected_loans


In [32]:
# Final summary statistics
print("=" * 70)
print("SILVER LAYER CLEANING SUMMARY")
print("=" * 70)

print("\nMapReduce Operations Used:")
print("  - map(): Data cleaning and transformation")
print("  - filter(): Remove invalid records")
print("  - flatMap(): Profile null values across columns")
print("  - reduceByKey(): Aggregate statistics by category")
print("  - aggregate(): Compute min/max/sum/count in single pass")
print("  - glom(): Analyze partition distribution")
print("  - repartition(): Optimize partitioning")

print("\nData Quality Improvements:")
print(f"  - Cleaned {len(KEY_COLUMNS)} key columns")
print(f"  - Standardized date formats")
print(f"  - Converted percentages and currency values")
print(f"  - Created binary target variable for ML")
print(f"  - Added derived features (fico_avg, loan_to_income)")

print("\nRecords:")
print(f"  - Bronze accepted: {accepted_rdd.count():,}")
print(f"  - Silver accepted: {verify_accepted.count():,}")
print(f"  - Records removed: {accepted_rdd.count() - verify_accepted.count():,}")

print("\nOutput Paths:")
print(f"  - {SILVER_ACCEPTED_PATH}")
print(f"  - {SILVER_REJECTED_PATH}")

print("=" * 70)

SILVER LAYER CLEANING SUMMARY

MapReduce Operations Used:
  - map(): Data cleaning and transformation
  - filter(): Remove invalid records
  - flatMap(): Profile null values across columns
  - reduceByKey(): Aggregate statistics by category
  - aggregate(): Compute min/max/sum/count in single pass
  - glom(): Analyze partition distribution
  - repartition(): Optimize partitioning

Data Quality Improvements:
  - Cleaned 27 key columns
  - Standardized date formats
  - Converted percentages and currency values
  - Created binary target variable for ML
  - Added derived features (fico_avg, loan_to_income)

Records:


  - Bronze accepted: 2,260,701
  - Silver accepted: 2,258,994


  - Records removed: 1,707

Output Paths:
  - ../data/medallion/silver/accepted_loans
  - ../data/medallion/silver/rejected_loans


In [33]:
# Unpersist cached RDDs to free memory
cleaned_rdd.unpersist()
filtered_rdd.unpersist()
optimized_rdd.unpersist()
cleaned_rejected_rdd.unpersist()

print("Cached RDDs unpersisted.")

Cached RDDs unpersisted.


## Next Steps

The Silver layer is complete. The data is now cleaned and ready for analytics and ML.

**Continue to:** `03_gold_serving.ipynb` for data serving using DataFrames, SQL, and MLlib.